In [25]:
import requests
import datetime
from urllib.parse import urlencode
import pandas as pd
import time
import base64


ModuleNotFoundError: No module named 'funciones_spoti'

# Acceso a la API de Spotify

In [2]:
file = open(f'../sptoken.txt')
client_secret = file.readlines()[0].strip('\n')

headers = {
    'Authorization': f'Bearer {client_secret}'
}

In [3]:
client_id = 'd48abffc774f4dd7af256702396c6ac3'

In [4]:
client_creds = f'{client_id}:{client_secret}'

client_cred_b64 = base64.b64encode(client_creds.encode())

In [5]:
# Se hace la request para que nos den el token y poder acceder a los datos

token_url = 'https://accounts.spotify.com/api/token'
method = 'POST'

token_data = {
        'grant_type': 'client_credentials' 
}

token_header = {
        'Authorization': f'Basic {client_cred_b64.decode()}'
}

In [6]:
r = requests.post(token_url, data=token_data, headers=token_header)
token_response_data = r.json()
access_token = token_response_data['access_token']
access_token

'BQAwQAgVz68IxQop-5tbclQMTprgPMxY_laXGF-oam23OR7MjeNY5tYWuzVp2_RPBNoTjcNOwPz_8ZW5jy0'

In [7]:
r

<Response [200]>

In [8]:
class SpotifyAPI(object):
    access_token = None
    access_token_expires = datetime.datetime.now()
    access_token_did_expire = True
    client_id = None
    client_secret = None
    token_url = "https://accounts.spotify.com/api/token"
    
    def __init__(self, client_id, client_secret, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.client_id = client_id
        self.client_secret = client_secret

    def get_client_credentials(self):
        """
        Returns a base64 encoded string
        """
        client_id = self.client_id
        client_secret = self.client_secret
        if client_secret == None or client_id == None:
            raise Exception("You must set client_id and client_secret")
        client_creds = f"{client_id}:{client_secret}"
        client_creds_b64 = base64.b64encode(client_creds.encode())
        return client_creds_b64.decode()
    
    def get_token_headers(self):
        client_creds_b64 = self.get_client_credentials()
        return {
            "Authorization": f"Basic {client_creds_b64}"
        }
    
    def get_token_data(self):
        return {
            "grant_type": "client_credentials"
        } 
    
    def perform_auth(self):
        token_url = self.token_url
        token_data = self.get_token_data()
        token_headers = self.get_token_headers()
        r = requests.post(token_url, data=token_data, headers=token_headers)
        if r.status_code not in range(200, 299):
            raise Exception("Could not authenticate client.")
            # return False
        data = r.json()
        now = datetime.datetime.now()
        access_token = data['access_token']
        expires_in = data['expires_in'] # seconds
        expires = now + datetime.timedelta(seconds=expires_in)
        self.access_token = access_token
        self.access_token_expires = expires
        self.access_token_did_expire = expires < now
        return True
    
    def get_access_token(self):
        token = self.access_token
        expires = self.access_token_expires
        now = datetime.datetime.now()
        if expires < now:
            self.perform_auth()
            return self.get_access_token()
        elif token == None:
            self.perform_auth()
            return self.get_access_token() 
        return token
    
    def get_resource_header(self):
        access_token = self.get_access_token()
        headers = {
            "Authorization": f"Bearer {access_token}"
        }
        return headers
        
        
    def get_resource(self, lookup_id, resource_type='playlists', version='v1'):
        endpoint = f"https://api.spotify.com/{version}/{resource_type}/{lookup_id}"
        headers = self.get_resource_header()
        r = requests.get(endpoint, headers=headers)
        if r.status_code not in range(200, 299):
            return {}
        return r.json()
    
    def get_playlist_tracks(self, query, version='v1', resource_type='playlists', item='tracks'):
        headers = self.get_resource_header()
        endpoint = f'https://api.spotify.com/{version}/{resource_type}/{query}/{item}'
        r = requests.get(endpoint, headers=headers)
        if r.status_code not in range(200, 299):
            return {}
        return r.json()
    
    def search_by_name(self, query, search_type='playlist'): # type
        headers = self.get_resource_header()
        endpoint = "https://api.spotify.com/v1/search"
        data = urlencode({"q": query, "type": search_type.lower()})
        lookup_url = f"{endpoint}?{data}"
        r = requests.get(lookup_url, headers=headers)
        if r.status_code not in range(200, 299):  
            return {}
        return r.json()

# Obtención de las playlist

In [9]:
headers = {
    "Authorization": f"Bearer {access_token}"
}

endpoint = "https://api.spotify.com/v1/playlist"
data = urlencode({"q": "Time", "type": "track"})
print(data)

lookup_url = f"{endpoint}?{data}"
r = requests.get(lookup_url, headers=headers)
print(r.status_code)

q=Time&type=track
404


In [10]:
# inicializamos el objeto de spotify

client_id = 'd48abffc774f4dd7af256702396c6ac3'
spotify = SpotifyAPI(client_id, client_secret)

In [11]:
def get_creator(dic):
    return dic['display_name']

In [12]:
def get_spoti_playlist(df):
    df.owner = list(map(get_creator, list(df.owner)))
    df = df[df['owner'] == 'Spotify']
    sxt_plst = df.loc[:0]
    return sxt_plst

In [13]:
#hay que hacer los endpoint


def get_df(pl_id):
    data = []

    sxt_tracks = spotify.get_playlist_tracks(pl_id)

    for i in range(len(sxt_tracks['items'])):
        art_lst = []
        sng_lst = []
        alb_lst = []
        dt_lst = []

        artista = sxt_tracks['items'][i]['track']['artists'][0]['name']
        art_lst.append(artista)

        cancion = sxt_tracks['items'][i]['track']['name']
        sng_lst.append(cancion)

        album = sxt_tracks['items'][i]['track']['album']['name']
        alb_lst.append(album)

        lanzamiento = sxt_tracks['items'][i]['track']['album']['release_date']
        dt_lst.append(lanzamiento)

        fila = []
        fila += art_lst + sng_lst + alb_lst + dt_lst

        data.append(fila)

    cols = ['Artista', 'Titulo', 'Album', 'Fecha']

    df = pd.DataFrame(data, columns=cols)
    return df

In [14]:
# Automatizo el proceso de obtencion del dataframe de las canciones

def get_every_track(nombre):
    
    sixtys = spotify.search_by_name(nombre, search_type="playlist")
    lst_plst = sixtys['playlists']['items']

    df = pd.DataFrame(lst_plst)

    sxt_plst = get_spoti_playlist(df)

    pl_id = str(sxt_plst['id']).split('    ')[-1].split('\n')[0]

    df_sxt = get_df(pl_id)
    return df_sxt

#  Años 60

In [15]:
df_sxt = get_every_track("All Out 60s")
df_sxt

,Artista,Titulo,Album,Fecha
0,The Temptations,My Girl,The Temptations Sing Smokey,1965-01-01
1,Percy Sledge,When a Man Loves a Woman,When a Man Loves a Woman,2018-05-05
2,The Rolling Stones,(I Can't Get No) Satisfaction - Mono Version,Hot Rocks (1964-1971),1971-12-20
3,Sam Cooke,(What A) Wonderful World,The Man Who Invented Soul,2000-09-26
4,Van Morrison,Brown Eyed Girl,Blowin' Your Mind!,1967-09
...,...,...,...,...
95,Ike & Tina Turner,River Deep - Mountain High,Tina!,2008-09-30
96,Herb Alpert & The Tijuana Brass,This Guy's In Love With You,The Beat Of The Brass,1968-05-01
97,Four Tops,Baby I Need Your Loving,Four Tops,1965-01-01
98,Bee Gees,Massachusetts (2008 Remastered LP Version),"Best Of Bee Gees, Vol. 1",1969-06-01


# Años 70

In [16]:
# Buscamos las más destacadas de los 70's

df_svn = get_every_track("All Out 70s")
df_svn

,Artista,Titulo,Album,Fecha
0,Elton John,"Rocket Man (I Think It's Going To Be A Long, L...",Honky Chateau,1972-05-19
1,Yusuf / Cat Stevens,Wild World,Tea For The Tillerman (Remastered),1970-11-23
2,Commodores,Easy,The Very Best Of The Commodores,1995-01-01
3,Eagles,Hotel California - 2013 Remaster,The Very Best of the Eagles (2013 Remaster),2003-10-21
4,Meat Loaf,Paradise By the Dashboard Light,Bat Out Of Hell,1977-10-21
...,...,...,...,...
95,America,Ventura Highway,Homecoming,1972-01-01
96,Sister Sledge,We Are Family,We Are Family,1979-02-15
97,Frankie Valli,Grease - 2007 Remaster,Frankie Valli...Is The Word,1978
98,Bill Withers,Ain't No Sunshine,Just As I Am,1971-05-01


# Años 80

In [17]:
# Buscamos las más destacadas de los 80's

df_egt = get_every_track("All Out 80s")
df_egt

,Artista,Titulo,Album,Fecha
0,a-ha,Take on Me,Hunting High and Low,1985-06-01
1,Phil Collins,A Groovy Kind Of Love,...Hits,1998-09-25
2,Cyndi Lauper,Time After Time,She's So Unusual: A 30th Anniversary Celebrati...,2014-03-28
3,Tina Turner,What's Love Got to Do with It,What's Love Got to Do with It?,1993
4,Bruce Springsteen,Dancing In the Dark,Born In The U.S.A.,1984-06-04
...,...,...,...,...
95,The B-52's,Love Shack,Cosmic Thing,1989-06-23
96,Dionne Warwick,"That's What Friends Are For (with Elton John, ...",80 Totally 80s Hits,2014-03-03
97,Bon Jovi,Livin' On A Prayer,Slippery When Wet,1986-08-16
98,Joe Cocker,Up Where We Belong,The Ultimate Collection 1968-2003,2003-12-01


# Años 90

In [18]:
# Buscamos las más destacads de los 90's

df_nns = get_every_track("All Out 90s")
df_nns

,Artista,Titulo,Album,Fecha
0,En Vogue,My Lovin' (You're Never Gonna Get It),Funky Divas,1992-03-24
1,Coolio,Gangsta's Paradise,Gangsta's Paradise,1995-11-07
2,Mariah Carey,Fantasy,Daydream,1995-10-03
3,Dr. Dre,Still D.R.E.,2001,1999-11-16
4,Janet Jackson,Together Again,The Velvet Rope,1997-10-07
...,...,...,...,...
95,Guns N' Roses,November Rain,Use Your Illusion I,1991-09-17
96,Blind Melon,No Rain,Blind Melon,1992-01-01
97,Oasis,Champagne Supernova,(What's the Story) Morning Glory?,1995-10-02
98,Fugees,"No Woman, No Cry",The Score (Expanded Edition),1996-02-13


# Años 00

In [19]:
# Buscamos las más destacadas de los 00's

df_zrs = get_every_track("All Out 2000s")
df_zrs

,Artista,Titulo,Album,Fecha
0,Mary J. Blige,Family Affair,No More Drama,2001-01-01
1,50 Cent,In Da Club,Get Rich Or Die Tryin',2003-02-06
2,Lady Gaga,Just Dance,Just Dance,2008-01-01
3,Gwen Stefani,The Sweet Escape,The Sweet Escape,2006-01-01
4,Shakira,Hips Don't Lie (feat. Wyclef Jean),"Oral Fixation, Vol. 2 (Expanded Edition)",2005-11-28
...,...,...,...,...
95,Beyoncé,If I Were a Boy,I AM...SASHA FIERCE - Platinum Edition,2008
96,Fergie,Big Girls Don't Cry (Personal),The Dutchess,2006-09-13
97,Nelly,Hot In Herre,Nellyville,2002-06-25
98,Timbaland,If We Ever Meet Again (Featuring Katy Perry),Shock Value II,2009-01-01
